In [1]:
import pandas as pd
from statistics import mean, median

In [2]:
project_list = ['rails.csv', 'gradle.csv', 'heroku.csv', 'jruby.csv', 'metasploit-framework.csv', 'cloudify.csv', 'vagrant.csv', 'rubinius.csv', 'open-build-service.csv', 'sonarqube.csv', 'loomio.csv', 'fog.csv', 'opal.csv', 'cloud_controller_ng.csv', 'puppet.csv', 'concerto.csv', 'sufia.csv', 'geoserver.csv', 'orbeon-forms.csv', 'graylog2-server.csv']

In [3]:
confidence = list(range(2,21,1))
algorithms = ['BATCH4', 'BATCHBISECT', 'BATCHSTOP4']
batch_sizes = [1, 2, 4, 8, 16]

In [4]:
result_rows = []

In [5]:
def str_to_list(s):
    if s == '[]':
        return [0]
    l = s[1:-1].split(', ')
    l = [int(x) for x in l]
    return l

In [6]:
for p in project_list:
    
    filename = p + '_sw.csv'
    data = pd.read_csv(filename)
    
    for alg in algorithms:
        for b in batch_sizes:
            
            if alg == 'BATCH4':
                if b != 4:
                    continue
            
            if alg == 'BATCHSTOP4':
                if b < 4:
                    continue
            
            for c in confidence:
                c_data = data[ (data['algorithm'] == alg) & (data['batch_size'] == b) & (data['confidence'] == c)]
                
                builds_reqd = 0
                delay_list = []
                total_length = 0
                total_actual_duration = 0
                total_build_duration = 0
                ci = []
            
                for i in range(len(c_data)):
                    row = c_data.iloc[i]
                    
                    builds_reqd = ((builds_reqd*total_length) + (row['builds_reqd'] * row['testall_size']))/(total_length + row['testall_size'])
                    total_length += row['testall_size']
                    total_actual_duration += row['actual_duration']
                    total_build_duration += row['build_duration']
                    delay_list.extend(str_to_list(row['delay_list']))
                    ci.extend(str_to_list(row['ci']))
                
                if len(delay_list) == 0:
                    delay_list = [0]
                final_row = [p, alg, b, c, builds_reqd, delay_list, median(delay_list), total_build_duration, total_actual_duration, total_length, ci]
                result_rows.append(final_row)
                

In [7]:
print(len(result_rows))

3420


In [8]:
df = pd.DataFrame(result_rows, columns=['project', 'algorithm', 'batch_size', 'confidence', 'builds_reqd', 'delay_list', 'median_delay', 'build_duration', 'actual_duration', 'testall_size', 'ci'])
df.to_csv('final_tr_results.csv')